In [62]:
from coffea.util import load
from coffea.analysis_tools import PackedSelection, Cutflow
from collections import namedtuple
import numpy as np

In [6]:
out1 = load('./outputs_old/FCCee/higgs/mH-recoil/mumu/mHrecoil_mumu.coffea')
out2 = load('./outputs_old/FCCee/higgs/mH-recoil/mumu/mHrecoil_mumu.coffea')

In [7]:
out1.keys()

dict_keys(['p8_ee_ZZ_ecm240', 'p8_ee_WW_ecm240', 'p8_ee_ZH_ecm240'])

In [43]:
c1 = out1['p8_ee_ZZ_ecm240']['cutflow']['sel0']
c2 = out1['p8_ee_ZZ_ecm240']['cutflow']['sel0']

In [51]:
c1.result()._asdict()

{'labels': ['initial',
  'No cut',
  'At least one Reco Particle',
  'Muon $p_T$ > 10 [GeV]',
  '$N_Z$',
  'Opp charge muons'],
 'nevonecut': [300000, 300000, 292809, 291002, 18038, 18003],
 'nevcutflow': [300000, 300000, 292809, 283811, 18038, 18003],
 'masksonecut': [array([ True,  True,  True, ...,  True,  True,  True]),
  array([ True,  True,  True, ...,  True,  True,  True]),
  array([ True,  True,  True, ...,  True,  True,  True]),
  array([False, False,  True, ..., False, False, False]),
  array([False, False,  True, ..., False, False, False])],
 'maskscutflow': [array([ True,  True,  True, ...,  True,  True,  True]),
  array([ True,  True,  True, ...,  True,  True,  True]),
  array([ True,  True,  True, ...,  True,  True,  True]),
  array([False, False,  True, ..., False, False, False]),
  array([False, False,  True, ..., False, False, False])]}

In [93]:
def add_cutflow(c1,c2):
    '''
    Add cutflow objects assuming they operate on non-overlaping sample regions
    '''
    r1 = c1.result()
    r2 = c2.result()


    if r1.labels == r2.labels :
        names = r1.labels
        names.remove('initial') # initial is added when Cutflow class is called, so removing it to preserve names list length
        names = names
        nevonecut = [a+b for a,b in zip(r1.nevonecut,r2.nevonecut)]
        nevcutflow = [a+b for a,b in zip(r1.nevcutflow,r2.nevcutflow)]
        masksonecut = [np.concatenate((a,b)) for a,b in zip(r1.masksonecut,r2.masksonecut)]
        maskscutflow = [np.concatenate((a,b)) for a,b in zip(r1.maskscutflow,r2.maskscutflow)]
        
    else:
        raise "The labels of the cutflow do not match!"
    return Cutflow(names, nevonecut, nevcutflow, masksonecut, maskscutflow, delayed_mode=False)

In [94]:
Cutflow.__add__ = add_cutflow #Monkey patch to enable the add method

In [95]:
(c1+c2).print()

Cutflow stats:
Cut No cut              :pass = 600000              cumulative pass = 600000              all = 600000              -- eff = 100.0 %                    -- cumulative eff = 100.0 %
Cut At least one Reco Particle:pass = 585618              cumulative pass = 585618              all = 600000              -- eff = 97.6 %                    -- cumulative eff = 97.6 %
Cut Muon $p_T$ > 10 [GeV]:pass = 582004              cumulative pass = 567622              all = 600000              -- eff = 97.0 %                    -- cumulative eff = 94.6 %
Cut $N_Z$               :pass = 36076               cumulative pass = 36076               all = 600000              -- eff = 6.0 %                    -- cumulative eff = 6.0 %
Cut Opp charge muons    :pass = 36006               cumulative pass = 36006               all = 600000              -- eff = 6.0 %                    -- cumulative eff = 6.0 %


In [96]:
(c1+c2).result()

CutflowResult(labels=['initial', 'No cut', 'At least one Reco Particle', 'Muon $p_T$ > 10 [GeV]', '$N_Z$', 'Opp charge muons'], nevonecut=[600000, 600000, 585618, 582004, 36076, 36006], nevcutflow=[600000, 600000, 585618, 567622, 36076, 36006], masksonecut=[array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True]), array([False, False,  True, ..., False, False, False]), array([False, False,  True, ..., False, False, False])], maskscutflow=[array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True]), array([False, False,  True, ..., False, False, False]), array([False, False,  True, ..., False, False, False])])

In [97]:
(c1+c2).yieldhist()

(Hist(Integer(0, 6, name='onecut'), storage=Double()) # Sum: 2439704.0,
 Hist(Integer(0, 6, name='cutflow'), storage=Double()) # Sum: 2425322.0,
 ['initial',
  'No cut',
  'At least one Reco Particle',
  'Muon $p_T$ > 10 [GeV]',
  '$N_Z$',
  'Opp charge muons'])

In [66]:
r1 = c1.result()
r2 = c2.result()

In [69]:
masksonecut = [np.concatenate((a,b)) for a,b in zip(r1.masksonecut,r2.masksonecut)]
len(r1.masksonecut)

5

In [70]:
r1.masksonecut[0].shape

(300000,)

In [71]:
len(masksonecut)

5

In [72]:
masksonecut[0].shape

(600000,)

In [82]:
r1.labels.remove('initial')

In [83]:
r1.labels

['No cut',
 'At least one Reco Particle',
 'Muon $p_T$ > 10 [GeV]',
 '$N_Z$',
 'Opp charge muons']

In [38]:
def accumulate(dicts):
    """
    Merges an array of dictionaries and adds up the values of common keys.

    Parameters:
    dicts (list): A list of dictionaries to be merged.

    Returns:
    dict: A dictionary with combined keys and values summed for common keys.
    """
    dict = {}

    for dictionary in dicts:
        for key, value in dictionary.items():
            if key in dict:
                dict[key] += value  # Add values if the key is common
            else:
                dict[key] = value  # Otherwise, add the new key-value pair 
    return dict